# Basic Machine Learning Workflow:

Import Libraries

In [ ]:
from fastai.vision.all import *

Get path to data

In [ ]:
path_data = Path.home()/'data'

In [ ]:
print(path_data)

/home/jupyter/data


In [ ]:
path_data.ls()

(#8) [Path('/home/jupyter/data/train_tp.csv'),Path('/home/jupyter/data/spectrogram_mel'),Path('/home/jupyter/data/train_fp.csv'),Path('/home/jupyter/data/spectrogram_frq'),Path('/home/jupyter/data/test'),Path('/home/jupyter/data/tfrecords'),Path('/home/jupyter/data/train'),Path('/home/jupyter/data/sample_submission.csv')]

In [ ]:
os.listdir(path_data)

FileNotFoundError: [Errno 2] No such file or directory: '~/data'

In [ ]:
path_data

In [ ]:
pd.read_csv()

In [ ]:
dataloader = ImageDataLoaders.